In [ ]:
import folium

from db.api import BaseDb
from folium.vector_layers import PolyLine, CircleMarker
from pyquadkey2 import quadkey
from pyquadkey2.quadkey import TileAnchor, QuadKey
from folium.vector_layers import PolyLine, CircleMarker

In [ ]:
class AreaDb(BaseDb):

    def __init__(self, folder='./data', file_name='qk-fences.db'):
        super().__init__(folder=folder, file_name=file_name)

    def get_level_qks(self, fence_id, level):
        sql = "select square_qk from geo_square where fence_id = ? and square_level = ?;"
        return [p[0] for p in self.query(sql, [fence_id, level])]

In [ ]:
def qk_to_str(qk, level):
    s = ""
    for i in range(level):
        shift = (level - i - 1) * 2
        mask = 3 << shift
        val = (qk & mask) >> shift
        s = s + chr(ord('0') + val)
    return s

In [ ]:
db = AreaDb()

In [ ]:
qks = {level: db.get_level_qks(1, level) for level in range(9, 18)}

In [ ]:
def create_map_polygon(xy, tooltip='',
                       color='#3388ff',
                       opacity=0.7,
                       fill_color='#3388ff',
                       fill_opacity=0.4, 
                       weight=3):
    points = [[x[0], x[1]] for x in xy]
    polygon = folium.vector_layers.Polygon(locations=points,
                                           tooltip=tooltip,
                                           fill=True,
                                           color=color,
                                           fill_color=fill_color,
                                           fill_opacity=fill_opacity,
                                           weight=weight,
                                           opacity=opacity)
    return polygon

In [ ]:
def create_map_quadkey(qk, tooltip='',
                       color='#3388ff',
                       opacity=0.7,
                       fill_color='#3388ff',
                       fill_opacity=0.4, 
                       weight=3):
    sw = qk.to_geo(anchor=TileAnchor.ANCHOR_SW)
    ne = qk.to_geo(anchor=TileAnchor.ANCHOR_NE)
    s, w = sw[0], sw[1]
    n, e = ne[0], ne[1]
    points = [[n, e], [s, e], [s, w], [n, w]]
    return create_map_polygon(points, tooltip=tooltip, 
                              color=color, opacity=opacity, 
                              fill_color=fill_color, fill_opacity=fill_opacity,
                              weight=weight)

In [ ]:
html_map = folium.Map(prefer_canvas=True, control_scale=True, 
                      max_zoom=18, tiles="cartodbpositron")

for level, quadkeys in qks.items():
    qk_list = [quadkey.from_str(qk_to_str(qk, level)) for qk in quadkeys]

    for qk in qk_list:
        create_map_quadkey(qk).add_to(html_map)
    
html_map